In [ ]:
# %tensorflow_version 1.x
# !pip install --upgrade opencv-python==3.4.2.17
import numpy as np
import tensorflow as tf
import tensorflow.keras
import tensorflow.keras.backend as K
# import os
from tensorflow.keras.datasets import fashion_mnist,mnist,cifar10
# import keras.backend as K
from tensorflow.keras.layers import Conv2D,Activation,BatchNormalization,UpSampling2D,Embedding,ZeroPadding2D, Input, Flatten, Dense, Reshape, LeakyReLU, Dropout,MaxPooling2D
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
from tensorflow.keras import regularizers

from tensorflow.keras.utils import Progbar
from keras.initializers import RandomNormal
import random
from sklearn.model_selection import train_test_split
# from keras.utils import np_utils
from tensorflow.keras import utils as np_utils



In [ ]:
nb_classes = 10
batch_size = 128
maxepoches = 250
learning_rate = 0.1
lr_decay = 1e-6
lr_drop = 20
def lr_scheduler(epoch):
    return learning_rate * (0.5 ** (epoch // lr_drop))
reduce_lr = tf.keras.callbacks.LearningRateScheduler(lr_scheduler)

In [ ]:
#Loading and splitting the dataset into train, validation and test


(X_Train, y_Train), (X_test, y_test) = cifar10.load_data()
X_train, X_val, y_train, y_val = train_test_split(X_Train, y_Train, test_size=0.20)
# convert y_train and y_test to categorical binary values 
Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_val = np_utils.to_categorical(y_val, nb_classes)
y_test = np_utils.to_categorical(y_test, nb_classes)

170508288/170498071 [==============================] - 3s 0us/step


In [ ]:
X_Train.shape

(50000, 32, 32, 3)

In [ ]:
# Reshape them to batch_size, width,height,#channels
X_train = X_train.reshape(40000, 32, 32, 3)
X_val = X_val.reshape(10000, 32, 32, 3)
X_test = X_test.reshape(10000, 32, 32, 3)

X_train = X_train.astype('float32')
X_val = X_val.astype('float32')
X_test = X_test.astype('float32')

# Normalize the values
X_train /= 255
X_val /= 255
X_test /= 255

In [ ]:
init=RandomNormal(mean=0,stddev=0.02)
input_shape = (32, 32, 3) # Input shape of each image
weight_decay = 0.0005
def build_model():
    # Build the network of vgg for 10 classes with massive dropout and weight decay as described in the paper.

    model = Sequential()

    model.add(Conv2D(64, (3, 3), padding='same',
                     input_shape=input_shape,kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.3))

    model.add(Conv2D(64, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())

    model.add(MaxPooling2D(pool_size=(2, 2),padding='same'))

    model.add(Conv2D(128, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.4))

    model.add(Conv2D(128, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())

    model.add(MaxPooling2D(pool_size=(2, 2),padding='same'))

    model.add(Conv2D(256, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.4))

    model.add(Conv2D(256, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.4))

    model.add(Conv2D(256, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())

    model.add(MaxPooling2D(pool_size=(2, 2),padding='same'))

    model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.4))

    model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.4))

    model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())

    model.add(MaxPooling2D(pool_size=(2, 2),padding='same'))

    model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.4))

    model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.4))

    model.add(Conv2D(512, (3, 3), padding='same',kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())

    model.add(MaxPooling2D(pool_size=(2, 2),padding='same'))
    model.add(Dropout(0.5))

    model.add(Flatten())
    model.add(Dense(256,kernel_regularizer=regularizers.l2(weight_decay), name='dense_1'))
    model.add(Activation('relu'))
    model.add(BatchNormalization())

    model.add(Dropout(0.5))
    model.add(Dense(10, name='dense_2'))
    model.add(Activation('softmax'))
    return model
teacher = build_model()

sgd = SGD(lr=learning_rate, decay=lr_decay, momentum=0.9, nesterov=True)

teacher.compile(loss='categorical_crossentropy',optimizer=sgd, metrics=['accuracy'])


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [ ]:
# teacher.fit(X_train,Y_train,batch_size=128,epochs=150,verbose=1,callbacks=[reduce_lr],validation_data=(X_val,Y_val))

In [ ]:
teacher.load_weights("Cifar10_Teacher.h5")

In [ ]:
# Y_test = np_utils.to_categorical(y_test, nb_classes)
loss, acc =teacher.evaluate(X_test, y_test, verbose=1)
loss, acc

313/313 [==============================] - 3s 9ms/step - loss: 0.8247 - accuracy: 0.8996


(0.834343671798706, 0.8992000222206116)

In [ ]:
#Collect the dense vector from the previous layer output and store it in a different model
teacher_WO_Softmax = Model(teacher.input, teacher.get_layer('dense_1').output)

In [ ]:
#Extracting dense representation from the teacher network
train_dense = teacher_WO_Softmax.predict(X_train)
val_dense = teacher_WO_Softmax.predict(X_val)

In [ ]:
#Splitting the training dense vector among N students(in this case 2)
s1Train=train_dense[:,:256]
s1Val=val_dense[:,:256]


In [ ]:
def define_model(name):
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(32, 32, 3), name=name))
    model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(0.2))
    model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(0.3))
    model.add(Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(0.4))
    model.add(Flatten())
    model.add(Dense(64, activation='relu', kernel_initializer='he_uniform'))
    model.add(Dropout(0.2))
    model.add(Dense(256, activation='relu', kernel_initializer='he_uniform',name='req'+name))

    model.compile(optimizer='nadam', loss='mse', metrics=['accuracy'])
    return model

student1 = define_model('s1')
student1.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
s1 (Conv2D)                  (None, 32, 32, 32)        896       
_________________________________________________________________
conv2d_32 (Conv2D)           (None, 32, 32, 32)        9248      
_________________________________________________________________
max_pooling2d_19 (MaxPooling (None, 16, 16, 32)        0         
_________________________________________________________________
dropout_23 (Dropout)         (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_33 (Conv2D)           (None, 16, 16, 128)       36992     
_________________________________________________________________
conv2d_34 (Conv2D)           (None, 16, 16, 128)       147584    
_________________________________________________________________
max_pooling2d_20 (MaxPooling (None, 8, 8, 128)        

TCN


In [ ]:
# Testing with LR
s1=define_model("s1")

opt=Adam(lr=0.0002, beta_1=0.9, beta_2=0.999, amsgrad=False)

In [ ]:
s1.compile(loss='mse', optimizer=opt)

s1.fit(X_train,s1Train,
          batch_size=256,
          epochs=80,
          verbose=1,
          validation_data=(X_val,s1Val))


Epoch 1/80
157/157 [==============================] - 3s 15ms/step - loss: 1.7760 - val_loss: 1.5999
Epoch 2/80
157/157 [==============================] - 2s 16ms/step - loss: 1.5822 - val_loss: 1.5609
Epoch 3/80
157/157 [==============================] - 2s 14ms/step - loss: 1.5335 - val_loss: 1.5122
Epoch 4/80
157/157 [==============================] - 3s 16ms/step - loss: 1.4935 - val_loss: 1.4719
Epoch 5/80
157/157 [==============================] - 2s 13ms/step - loss: 1.4664 - val_loss: 1.4405
Epoch 6/80
157/157 [==============================] - 3s 17ms/step - loss: 1.4447 - val_loss: 1.4194
Epoch 7/80
157/157 [==============================] - 2s 14ms/step - loss: 1.4272 - val_loss: 1.4062
Epoch 8/80
157/157 [==============================] - 3s 18ms/step - loss: 1.4168 - val_loss: 1.3989
Epoch 9/80
157/157 [==============================] - 2s 14ms/step - loss: 1.4060 - val_loss: 1.3892
Epoch 10/80
157/157 [==============================] - 2s 15ms/step - loss: 1.4023 - val_lo

In [ ]:
output=s1.get_layer("reqs1").output

output=Activation('relu')(output)
output2=Dropout(0.5)(output) # For reguralization
output3=Dense(10,activation="softmax", name="d1")(output2)

mm=Model(s1.get_layer("s1").input, output3)
my_weights=teacher.get_layer('dense_2').get_weights()
mm.get_layer('d1').set_weights(my_weights)

In [ ]:
i=0
for l in mm.layers[:len(mm.layers)-2]:
    l.trainable=False
#     print(l)

mm.compile(loss='categorical_crossentropy',
              optimizer=Adam(learning_rate=0.0002),
              metrics=['accuracy'])

# Without finetune
batch_size = 256
mm_history=mm.fit([X_train], Y_train,
          batch_size=batch_size,
          epochs=50,
          verbose=1,
          validation_data=([X_val], Y_val))

Epoch 1/50
157/157 [==============================] - 2s 9ms/step - loss: 1.4346 - accuracy: 0.7275 - val_loss: 1.0192 - val_accuracy: 0.8067
Epoch 2/50
157/157 [==============================] - 2s 10ms/step - loss: 1.1096 - accuracy: 0.7642 - val_loss: 0.8819 - val_accuracy: 0.8121
Epoch 3/50
157/157 [==============================] - 1s 8ms/step - loss: 1.0334 - accuracy: 0.7646 - val_loss: 0.7984 - val_accuracy: 0.8132
Epoch 4/50
157/157 [==============================] - 2s 11ms/step - loss: 0.9726 - accuracy: 0.7635 - val_loss: 0.7389 - val_accuracy: 0.8136
Epoch 5/50
157/157 [==============================] - 1s 9ms/step - loss: 0.9231 - accuracy: 0.7607 - val_loss: 0.7023 - val_accuracy: 0.8129
Epoch 6/50
157/157 [==============================] - 1s 8ms/step - loss: 0.9009 - accuracy: 0.7614 - val_loss: 0.6702 - val_accuracy: 0.8140
Epoch 7/50
157/157 [==============================] - 2s 10ms/step - loss: 0.8630 - accuracy: 0.7620 - val_loss: 0.6552 - val_accuracy: 0.8124
Epo